In [4]:
from LLms.ollamaLllm import apiCall as apiCallOllama
from LLms.gptLLm import apiCall as apiCallGpt
import random
import pandas as pd
import os
import time

## Import films data

In [5]:
file_path = 'Datasets/netflix dataset/netflix_titles.csv'
movieDf = pd.read_csv(file_path)

## Structured descriptions of movies

In [7]:
numberOfMovie = 0
numberOfFactsCategories = 0
numberOfGoodFactsCategories = 0
GPTTemp = 0
def structured_description():
    # Request to generate the structured descriptions, given a context and a list of facts about a movie for numberOfMovie
    column_names = movieDf.columns
    listOfDescriptions = []
    endInstruction = []
    random_numbers = random.sample(range(0, 8800), numberOfMovie)

    # We make a list with every fact and a text with every fact except the last one. This text will be used as a prompt. Also if a fact include "nan", we don't use it
    # every thing is stored in listOfDescriptions with a length of numberOfMovie
    # listOfDescriptions[i][0] is the structured description of the movie i 
    # listOfDescriptions[i][1] is the list of facts
    # listOfDescriptions[i][2] is the text with the facts except the last one
    numberOfFacts1 = 0
    for i in random_numbers:
        instruction = ""
        facts = []
        randomizedRange= random.sample(range(0, numberOfFactsCategories), numberOfFactsCategories)
        t = 0 # track the number of "good" facts (facts that will be given to the llm), when t is more than numberOfGoodFacts, the facts won't be given to the llm
        for j in randomizedRange:
            tempFact = str(movieDf[column_names[j+1]][i])
            tempFact = column_names[j+1] + " = " + tempFact + " \n"
            if " nan " in tempFact : 
                continue
            facts.append(tempFact)
            numberOfFacts1 +=1
            if t >= numberOfGoodFactsCategories : 
                continue
            t+= 1
            instruction = instruction + tempFact
            
        result = apiCallGpt(instruction,GPTTemp)
        listOfDescriptions.append([result, facts, instruction])
        
        
    #Display a number (x) of random descriptions with the informations that were given for generating it
    # x = min(5, numberOfMovie)
    # random_numbers = random.sample(range(0, numberOfMovie), x)
    # for i in random_numbers: 
    #     print("Instructions : "+ listOfDescriptions[i][2])
    #     print("Structured description : "+ listOfDescriptions[i][0] + "\n")
    return listOfDescriptions


## Use smaller llm to identify facts and wheter the fact is correct or not in each descriptions
## Use few shot prompting to improve the format of the answer 

In [8]:
ollamaTemp = 0
model = 0
def identify_facts(listOfDescriptions):
    # Request to identifie facts and accuracy, given a context, a description and a list of facts about a movie for numberOfMovie
    listOfFoundFacts = []
    numberOfFacts2 = 0
    numberOfRightFacts = 0
    numberOfbadFacts = 0
    scoreFalseNegative = 0
    scoreTruePositive = 0
    scoreFalsePositive = 0
    scoreTrueNegative = 0
    scoreBadOutput = 0
    for i in range (numberOfMovie):
        if i % 10 == 0:
            print(i)
        instruction = ""
        listOfFoundFacts.append([])
        for j in range(len(listOfDescriptions[i][1])):
            numberOfFacts2 += 1
            instruction = "The fact is : [" + listOfDescriptions[i][1][j]+"] and the description is [" + listOfDescriptions[i][0] + "]"
            t0 = time.time()
            result = apiCallOllama(instruction,ollamaTemp, model).lower()
            t1 = time.time()
            print(f"One api call done in {t1-t0} seconds")
            if j >= numberOfGoodFactsCategories :
                # the facts after numberOfGoodFacts should not be in the description
                numberOfbadFacts += 1 
                if "correct" in result: 
                    scoreFalsePositive +=1
                    listOfFoundFacts[i].append( [listOfDescriptions[i][1][j], listOfDescriptions[i][0], result, "FP  -  FalsePositive"])
                elif "wrong" in result: 
                    scoreTrueNegative += 1
                    listOfFoundFacts[i].append( [listOfDescriptions[i][1][j], listOfDescriptions[i][0], result, "TN  -  TrueNegative"])
                else : 
                    scoreBadOutput += 1
                    listOfFoundFacts[i].append( [listOfDescriptions[i][1][j], listOfDescriptions[i][0], result, "!!! BAD OUTPUT !!!"])
            else :
                # every fact here is supposed to be found
                numberOfRightFacts +=1
                if "correct" in result: 
                    scoreTruePositive +=1
                    listOfFoundFacts[i].append( [listOfDescriptions[i][1][j], listOfDescriptions[i][0], result, "TP  -  TruePositive"])
                elif "wrong" in result: 
                    scoreFalseNegative += 1
                    listOfFoundFacts[i].append( [listOfDescriptions[i][1][j], listOfDescriptions[i][0], result, "FN  -  FalseNegative"])
                else : 
                    scoreBadOutput += 1
                    listOfFoundFacts[i].append( [listOfDescriptions[i][1][j], listOfDescriptions[i][0], result, "!!! BAD OUTPUT !!!"])

    # make the metrics                
                    
    Accuracy = (scoreTrueNegative + scoreTruePositive) / numberOfFacts2 * 100
    precision = scoreTruePositive / (scoreFalsePositive + scoreTruePositive ) * 100
    recall = scoreTruePositive / numberOfRightFacts * 100
    specificity  = scoreTrueNegative / numberOfbadFacts * 100
    falsePositiveRate = scoreFalsePositive / (scoreFalsePositive + scoreTrueNegative ) *100
    fMeasure = 2 * precision * recall /( precision + recall)


    # write all result in a txt file
    result_file_number = int(open("fileNumber.txt","r").readline())
    result_file_number += 1
    open("fileNumber.txt","w").writelines(str(result_file_number))
    fileName = "results/result" + str(result_file_number)+".txt"
    with open(fileName,"w") as fout:
        fout.writelines("techniques  = basics")
        fout.writelines("numberOfAgent = 1 \n")
        fout.writelines("Model =  " + model + "\n")
        fout.writelines("GPT temp =  " + str(GPTTemp) + "\n")
        fout.writelines("ollama temp =  " + str(ollamaTemp) + "\n")
        fout.writelines("Accuracy =  " + str(Accuracy) + "\n")
        fout.writelines("precision =  " + str(precision) + "\n")
        fout.writelines("recall =  " + str(recall) + "\n")
        fout.writelines("specificity =  " + str(specificity) + "\n")
        fout.writelines("falsePositiveRate =  " + str(falsePositiveRate) + "\n")
        fout.writelines("fMeasure =  " + str(fMeasure) + "\n \n \n")
        fout.writelines("NumberOfMovies  = "+ str(numberOfMovie) + "\n")
        fout.writelines("NumberOfFacts  = "+ str(numberOfFacts2) + "\n")
        fout.writelines("numberOfFactsCategories  = "+ str(numberOfFactsCategories) + "\n")
        fout.writelines("numberOfGoodFactsCategories  = "+ str(numberOfGoodFactsCategories) + "\n")
        fout.writelines("numberOfRightFacts =  " + str(numberOfRightFacts) + " and numberOfbadFacts  = "+ str(numberOfbadFacts) + "\n \n \n")
        fout.writelines("scoreBadOutput =  " + str(scoreBadOutput) + "\n")
        fout.writelines("scoreTruePositive =  " + str(scoreTruePositive) + "\n")
        fout.writelines("scoreTrueNegative =  " + str(scoreTrueNegative) + "\n")
        fout.writelines("scoreFalsePositive =  " + str(scoreFalsePositive) + "\n")
        fout.writelines("scoreFalseNegative =  " + str(scoreFalseNegative) + "\n \n")
        for i in range(numberOfMovie):
            try :
                fout.writelines(listOfDescriptions[i][2]+"\n")
            except UnicodeEncodeError as e:
                pass
            
            for j in range(len(listOfFoundFacts[i])):
                fout.writelines(listOfFoundFacts[i][j][3]+"\n")
                fout.writelines(listOfFoundFacts[i][j][2]+"\n")
                try :
                    fout.writelines("fact : "+ listOfFoundFacts[i][j][0]+"\n")
                except UnicodeEncodeError as e:
                    pass
                try : 
                    fout.writelines( listOfFoundFacts[i][j][1]+" \n \n \n")
                except UnicodeEncodeError as e:
                    pass
            fout.writelines("\n \n")


## Small llm + parallel agents

In [5]:
ollamaTemp = 0
model = 0
def identify_facts_parallel_agents(listOfDescriptions, numberOfAgent):
    # Request to identifie facts and accuracy, given a context, a description and a list of facts about a movie for numberOfMovie
    listOfFoundFacts = []
    numberOfFacts2 = 0
    numberOfRightFacts = 0
    numberOfbadFacts = 0
    scoreFalseNegative = 0
    scoreTruePositive = 0
    scoreFalsePositive = 0
    scoreTrueNegative = 0
    scoreBadOutput = 0
    for i in range (numberOfMovie):
        if i % 10 == 0:
            print(i)
        instruction = ""
        listOfFoundFacts.append([])
        for j in range(len(listOfDescriptions[i][1])):
            result = []
            numberOfFacts2 += 1
            instruction = "The fact is : [" + listOfDescriptions[i][1][j]+"] and the description is [" + listOfDescriptions[i][0] + "]"
            for agent in range(numberOfAgent):
                result.append(apiCallOllama(instruction,ollamaTemp, model).lower())
                
                
                
            numberOfCorrect = 0
            numberOfWrong = 0
            for k in result:
                if "correct" in k:
                    numberOfCorrect +=1
                elif "wrong" in k:
                    numberOfWrong +=1
                else :
                    pass
            if j >= numberOfGoodFactsCategories :
                # the facts after numberOfGoodFacts should not be in the description
                numberOfbadFacts += 1 
                if numberOfCorrect >= numberOfWrong and numberOfCorrect > 0: 
                    scoreFalsePositive +=1
                    listOfFoundFacts[i].append( [listOfDescriptions[i][1][j], listOfDescriptions[i][0], result, "FP  -  FalsePositive"])
                elif numberOfCorrect < numberOfWrong: 
                    scoreTrueNegative += 1
                    listOfFoundFacts[i].append( [listOfDescriptions[i][1][j], listOfDescriptions[i][0], result, "TN  -  TrueNegative"])
                else : 
                    scoreBadOutput += 1
                    listOfFoundFacts[i].append( [listOfDescriptions[i][1][j], listOfDescriptions[i][0], result, "!!! BAD OUTPUT !!!"])
            else :
                # every fact here is supposed to be found
                numberOfRightFacts +=1
                if numberOfCorrect >= numberOfWrong and numberOfCorrect > 0: 
                    scoreTruePositive +=1
                    listOfFoundFacts[i].append( [listOfDescriptions[i][1][j], listOfDescriptions[i][0], result, "TP  -  TruePositive"])
                elif numberOfCorrect < numberOfWrong: 
                    scoreFalseNegative += 1
                    listOfFoundFacts[i].append( [listOfDescriptions[i][1][j], listOfDescriptions[i][0], result, "FN  -  FalseNegative"])
                else : 
                    scoreBadOutput += 1
                    listOfFoundFacts[i].append( [listOfDescriptions[i][1][j], listOfDescriptions[i][0], result, "!!! BAD OUTPUT !!!"])

    # make the metrics                
                    
    Accuracy = (scoreTrueNegative + scoreTruePositive) / numberOfFacts2 * 100
    precision = scoreTruePositive / (scoreFalsePositive + scoreTruePositive ) * 100
    recall = scoreTruePositive / numberOfRightFacts * 100
    specificity  = scoreTrueNegative / numberOfbadFacts * 100
    falsePositiveRate = scoreFalsePositive / (scoreFalsePositive + scoreTrueNegative ) *100
    fMeasure = 2 * precision * recall /( precision + recall)


    # write all result in a txt file
    result_file_number = int(open("fileNumber.txt","r").readline())
    result_file_number += 1
    open("fileNumber.txt","w").writelines(str(result_file_number))
    fileName = "results/result" + str(result_file_number)+".txt"
    with open(fileName,"w") as fout:
        fout.writelines("techniques  = basic + parallel agents \n")
        fout.writelines("numberOfAgent =  " + str(numberOfAgent) + "\n")
        fout.writelines("Model =  " + model + "\n")
        fout.writelines("GPT temp =  " + str(GPTTemp) + "\n")
        fout.writelines("ollama temp =  " + str(ollamaTemp) + "\n")
        fout.writelines("Accuracy =  " + str(Accuracy) + "\n")
        fout.writelines("precision =  " + str(precision) + "\n")
        fout.writelines("recall =  " + str(recall) + "\n")
        fout.writelines("specificity =  " + str(specificity) + "\n")
        fout.writelines("falsePositiveRate =  " + str(falsePositiveRate) + "\n")
        fout.writelines("fMeasure =  " + str(fMeasure) + "\n \n \n")
        fout.writelines("NumberOfMovies  = "+ str(numberOfMovie) + "\n")
        fout.writelines("NumberOfFacts  = "+ str(numberOfFacts2) + "\n")
        fout.writelines("numberOfFactsCategories  = "+ str(numberOfFactsCategories) + "\n")
        fout.writelines("numberOfGoodFactsCategories  = "+ str(numberOfGoodFactsCategories) + "\n")
        fout.writelines("numberOfRightFacts =  " + str(numberOfRightFacts) + " and numberOfbadFacts  = "+ str(numberOfbadFacts) + "\n \n \n")
        fout.writelines("scoreBadOutput =  " + str(scoreBadOutput) + "\n")
        fout.writelines("scoreTruePositive =  " + str(scoreTruePositive) + "\n")
        fout.writelines("scoreTrueNegative =  " + str(scoreTrueNegative) + "\n")
        fout.writelines("scoreFalsePositive =  " + str(scoreFalsePositive) + "\n")
        fout.writelines("scoreFalseNegative =  " + str(scoreFalseNegative) + "\n \n")
        for i in range(numberOfMovie):
            try :
                fout.writelines(listOfDescriptions[i][2]+"\n")
            except UnicodeEncodeError as e:
                pass
            
            for j in range(len(listOfFoundFacts[i])):
                fout.writelines(listOfFoundFacts[i][j][3]+"\n")
                fout.writelines(str(listOfFoundFacts[i][j][2])+"\n")
                try :
                    fout.writelines("fact : "+ listOfFoundFacts[i][j][0]+"\n")
                except UnicodeEncodeError as e:
                    pass
                try : 
                    fout.writelines( listOfFoundFacts[i][j][1]+" \n \n \n")
                except UnicodeEncodeError as e:
                    pass
            fout.writelines("\n \n")

## Format for chatgpt and local llm request

### Run for differents ollama temp

In [10]:
# "llama3_fact" , "mistral_fact", "gemma_fact"
model = "llama3_fact"
GPTTemp = 0.3
ollamaTemps = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
numberOfMovie = 5
numberOfFactsCategories = 10
numbersOfGoodFactsCategories = [5]
listOfDescriptions = structured_description()
for i in ollamaTemps:
    for j in numbersOfGoodFactsCategories :
        numberOfGoodFactsCategories = j
        ollamaTemp = i
        identify_facts(listOfDescriptions)
        print(str(i) +" is done")


0
One api call done in 6.906246662139893 seconds
One api call done in 5.535760879516602 seconds
One api call done in 5.803540468215942 seconds
One api call done in 5.7218897342681885 seconds
One api call done in 5.838862180709839 seconds


KeyboardInterrupt: 

### Run for differents numbers of good facts, 3 times, for each models

In [ ]:
# "llama3_fact" , "mistral_fact", "gemma_fact"
models = ["llama3_fact","gemma_fact"]
GPTTemp = 0.3
ollamaTemps = [0.3]
numberOfMovie = 50
numberOfFactsCategories = 10
numbersOfGoodFactsCategories = [1,3,5,7,9]
for m in range(3) :
    for k in models :
        model = k
        for i in ollamaTemps:
            for j in numbersOfGoodFactsCategories :
                numberOfGoodFactsCategories = j
                listOfDescriptions = structured_description()
                ollamaTemp = i
                identify_facts(listOfDescriptions)
    print("itération totale : " +str(m)+" done")

### Run for parallel agents, for multiple agents, for each models

In [6]:
# "llama3_fact" , "mistral_fact", "gemma_fact"
models = ["llama3_fact","gemma_fact"]
GPTTemp = 0.3
ollamaTemps = [0.3]
numberOfMovie = 50
numberOfFactsCategories = 10
numbersOfGoodFactsCategories = [5]
numberOfAgents = [1,3]
for m in range(3) :
    listOfDescriptions = structured_description()
    for k in models :
        model = k
        for i in ollamaTemps:
            ollamaTemp = i
            for n in numberOfAgents : 
                numberOfAgent = n
                for j in numbersOfGoodFactsCategories :
                    numberOfGoodFactsCategories = j
                    t0 = time.time()
                    identify_facts_parallel_agents(listOfDescriptions, numberOfAgent)
                    t1 = time.time()
                    print(f"Temps pris : {t1-t0}")
                print("numberOfAgent : " +str(n)+" done")
    print("itération totale : " +str(m)+" done")

0
10
20
30
40
Temps pris : 1219.6009809970856
numberOfAgent : 1 done
0
10
20
30
40
Temps pris : 3205.487319469452
numberOfAgent : 3 done
0
10
20
30
40
Temps pris : 1200.5958716869354
numberOfAgent : 1 done
0
10
20
30
40
Temps pris : 3259.908574819565
numberOfAgent : 3 done
itération totale : 0 done
0
10
20
30
40
Temps pris : 1194.779622077942
numberOfAgent : 1 done
0
10
20
30
40
Temps pris : 3178.349246263504
numberOfAgent : 3 done
0
10
20
30
40
Temps pris : 1225.4274461269379
numberOfAgent : 1 done
0
10
20
30
40
Temps pris : 3213.4239802360535
numberOfAgent : 3 done
itération totale : 1 done
0
10
20
30
40
Temps pris : 1177.9410665035248
numberOfAgent : 1 done
0
10
20
30
40
Temps pris : 3138.124322414398
numberOfAgent : 3 done
0
10
20
30
40
Temps pris : 1204.5686073303223
numberOfAgent : 1 done
0
10
20
30
40
Temps pris : 3176.698557138443
numberOfAgent : 3 done
itération totale : 2 done


### Test the api

In [ ]:
instruction =["","","","",""]
instruction[0] = 'the fact is : [cast = Beth Chalmers, David Holt, Marcel McCalla, Teresa Gallagher] and the description is ["Numberblocks" is a British TV show that was added on September 15, 2021. This educational show is targeted towards young children as it has a TV-Y rating. The show consists of 6 seasons and features a cast including Beth Chalmers, David Holt, Marcel McCalla, and Teresa Gallagher. "Numberblocks" likely focuses on teaching children basic math concepts in a fun and engaging way, using colorful characters and entertaining stories to make learning numbers enjoyable.]'
instruction[1] = 'the fact is : [cast = Tom Brady] and the description is ["Numberblocks" is a British TV show that was added on September 15, 2021. This educational show is targeted towards young children as it has a TV-Y rating. The show consists of 6 seasons and features a cast including Beth Chalmers, David Holt, Marcel McCalla, and Teresa Gallagher. "Numberblocks" likely focuses on teaching children basic math concepts in a fun and engaging way, using colorful characters and entertaining stories to make learning numbers enjoyable.]'
instruction[2] = 'the fact is : [movie type = tv show] and the description is ["Numberblocks" is a British TV show that was added on September 15, 2021. This educational show is targeted towards young children as it has a TV-Y rating. The show consists of 6 seasons and features a cast including Beth Chalmers, David Holt, Marcel McCalla, and Teresa Gallagher. "Numberblocks" likely focuses on teaching children basic math concepts in a fun and engaging way, using colorful characters and entertaining stories to make learning numbers enjoyable.]'
instruction[3] = 'the fact is : [director = S. Ahmrad] and the description is ["Numberblocks" is a British TV show that was added on September 15, 2021. This educational show is targeted towards young children as it has a TV-Y rating. The show consists of 6 seasons and features a cast including Beth Chalmers, David Holt, Marcel McCalla, and Teresa Gallagher. "Numberblocks" likely focuses on teaching children basic math concepts in a fun and engaging way, using colorful characters and entertaining stories to make learning numbers enjoyable.]'
instruction[4] = 'the fact is : [Title = Block] and the description is ["Numberblocks" is a British TV show that was added on September 15, 2021. This educational show is targeted towards young children as it has a TV-Y rating. The show consists of 6 seasons and features a cast including Beth Chalmers, David Holt, Marcel McCalla, and Teresa Gallagher. "Numberblocks" likely focuses on teaching children basic math concepts in a fun and engaging way, using colorful characters and entertaining stories to make learning numbers enjoyable.]'
reponse = ["correct","wrong","correct","wrong",'wrong']
for i in range(5): 
    result = apiCallOllama(instruction[i],ollamaTemp, model)
    print(result)
print(reponse)


correct
wrong
correct
wrong
wrong
['correct', 'wrong', 'correct', 'wrong', 'wrong']
